# Acrobot

## A2C Agent 

In [ ]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "Acrobot-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "Acrobot-v1":
    gamma = 0.95
    lr = 7e-4

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

In [ ]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (KL) 

In [48]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = "Acrobot-v1"
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent

# When the learning rate is large, policy neural network can overflow and lead to NaNs. 
# A possible fix is to reduce lr or increase beta to lower the learning rate.

if kl_env == "Acrobot-v1":
    gamma = 0.95
    lr = 1e-2
    beta = 0.8

agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration
    beta += np.random.random()*0.1
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward  <= -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2
        
    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= -490):
        agent = DRTRPOAgent(env, gamma, lr)

    policy_loss = agent.compute_policy_loss_kl(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

Episode 0: -448.0
Episode 1: -464.0
Episode 2: -484.6666666666667
Episode 3: -445.3333333333333
Episode 4: -456.0
Episode 5: -426.3333333333333
Episode 6: -500.0
Episode 7: -254.33333333333334
Episode 8: -220.33333333333334
Episode 9: -233.33333333333334
Episode 10: -199.0
Episode 11: -181.66666666666666
Episode 12: -191.33333333333334
Episode 13: -179.66666666666666
Episode 14: -291.0
Episode 15: -144.33333333333334
Episode 16: -170.33333333333334
Episode 17: -159.66666666666666
Episode 18: -173.33333333333334
Episode 19: -189.0
Episode 20: -193.33333333333334
Episode 21: -221.66666666666666
Episode 22: -162.33333333333334
Episode 23: -166.66666666666666
Episode 24: -175.33333333333334
Episode 25: -152.66666666666666
Episode 26: -231.66666666666666
Episode 27: -180.0
Episode 28: -169.66666666666666
Episode 29: -146.0
Episode 30: -184.0
Episode 31: -209.0
Episode 32: -168.66666666666666
Episode 33: -157.0
Episode 34: -150.0
Episode 35: -166.33333333333334
Episode 36: -184.0
Episode 37:

Episode 288: -189.33333333333334
Episode 289: -200.0
Episode 290: -171.66666666666666
Episode 291: -190.66666666666666
Episode 292: -213.0
Episode 293: -178.33333333333334
Episode 294: -167.33333333333334
Episode 295: -175.66666666666666
Episode 296: -198.0
Episode 297: -160.0
Episode 298: -187.66666666666666
Episode 299: -167.0


In [49]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Wasserstein)

In [40]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "Acrobot-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "Acrobot-v1":
    gamma = 0.95
    lr = 1e-2
    
agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
    
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration 
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward < -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2

    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= -490):
        agent = DRTRPOAgent(env, gamma, lr)
        
    total_adv_diff += max(abs(state_adv[1] - state_adv[0]), abs(state_adv[2] - state_adv[0]), abs(state_adv[2] - state_adv[1]))
    beta = total_adv_diff/episode
    beta_l = 0.6
    beta_s = 0.1
    policy_loss = agent.compute_policy_loss_wass(first_state, state_adv, beta_s, beta_l)

    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -500.0
Episode 5: -500.0
Episode 6: -500.0
Episode 7: -500.0
Episode 8: -175.33333333333334
Episode 9: -149.0
Episode 10: -127.66666666666667
Episode 11: -117.0
Episode 12: -154.66666666666666
Episode 13: -164.66666666666666
Episode 14: -137.66666666666666
Episode 15: -121.33333333333333
Episode 16: -101.0
Episode 17: -125.0
Episode 18: -217.66666666666666
Episode 19: -150.33333333333334
Episode 20: -141.0
Episode 21: -137.66666666666666
Episode 22: -113.33333333333333
Episode 23: -123.33333333333333
Episode 24: -130.66666666666666
Episode 25: -165.33333333333334
Episode 26: -182.33333333333334
Episode 27: -142.0
Episode 28: -121.66666666666667
Episode 29: -149.0
Episode 30: -161.66666666666666
Episode 31: -119.0
Episode 32: -130.33333333333334
Episode 33: -129.66666666666666
Episode 34: -129.0
Episode 35: -164.66666666666666
Episode 36: -157.33333333333334
Episode 37: -168.66666666666666
Episode 38: -14

Episode 290: -141.33333333333334
Episode 291: -256.3333333333333
Episode 292: -123.66666666666667
Episode 293: -140.0
Episode 294: -172.33333333333334
Episode 295: -129.33333333333334
Episode 296: -162.0
Episode 297: -142.0
Episode 298: -154.0
Episode 299: -119.66666666666667


In [41]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')